In [6]:
# encoding=utf-8
import numpy as np
import scipy.io
import scipy.linalg
import sklearn.metrics
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import gdal
import os
import cv2 
from sklearn.cluster import KMeans 
from sklearn.model_selection import train_test_split
from sklearn import tree, svm, naive_bayes,neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier,  GradientBoostingClassifier

In [2]:
def kernel(ker, X1, X2, gamma):
    K = None
    if not ker or ker == 'primal':
        K = X1
    elif ker == 'linear':
        if X2 is not None:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1).T, np.asarray(X2).T)
        else:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1).T)
    elif ker == 'rbf':
        if X2 is not None:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1).T, np.asarray(X2).T, gamma)
        else:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1).T, None, gamma)
    return K


class TCA:
    def __init__(self, kernel_type='primal', dim=30, lamb=1, gamma=1):
        '''
        Init func
        :param kernel_type: kernel, values: 'primal' | 'linear' | 'rbf'
        :param dim: dimension after transfer
        :param lamb: lambda value in equation
        :param gamma: kernel bandwidth for rbf kernel
        '''
        self.kernel_type = kernel_type
        self.dim = dim
        self.lamb = lamb
        self.gamma = gamma

    def fit(self, Xs, Xt):
        '''
        Transform Xs and Xt
        :param Xs: ns * n_feature, source feature
        :param Xt: nt * n_feature, target feature
        :return: Xs_new and Xt_new after TCA
        '''
        X = np.hstack((Xs.T, Xt.T))
        X /= np.linalg.norm(X, axis=0)
        m, n = X.shape
        ns, nt = len(Xs), len(Xt)
        e = np.vstack((1 / ns * np.ones((ns, 1)), -1 / nt * np.ones((nt, 1))))
        M = e * e.T
        M = M / np.linalg.norm(M, 'fro')
        H = np.eye(n) - 1 / n * np.ones((n, n))
        K = kernel(self.kernel_type, X, None, gamma=self.gamma)
        n_eye = m if self.kernel_type == 'primal' else n
        a, b = np.linalg.multi_dot([K, M, K.T]) + self.lamb * np.eye(n_eye), np.linalg.multi_dot([K, H, K.T])
        w, V = scipy.linalg.eig(a, b)
        ind = np.argsort(w)
        A = V[:, ind[:self.dim]]
        Z = np.dot(A.T, K)
        Z /= np.linalg.norm(Z, axis=0)
        Xs_new, Xt_new = Z[:, :ns].T, Z[:, ns:].T
        return Xs_new, Xt_new

    def fit_predict(self, Xs, Ys, Xt, Yt):
        '''
        Transform Xs and Xt, then make predictions on target using 1NN
        :param Xs: ns * n_feature, source feature
        :param Ys: ns * 1, source label
        :param Xt: nt * n_feature, target feature
        :param Yt: nt * 1, target label
        :return: Accuracy and predicted_labels on the target domain
        '''
        Xs_new, Xt_new = self.fit(Xs, Xt)
        clf = KNeighborsClassifier(n_neighbors=1)
        clf.fit(Xs_new, Ys.ravel())
        y_pred = clf.predict(Xt_new)
        acc = sklearn.metrics.accuracy_score(Yt, y_pred)
        return acc, y_pred


# if __name__ == '__main__':
#     domains = ['caltech.mat', 'amazon.mat', 'webcam.mat', 'dslr.mat']
#     for i in [2]:
#         for j in [3]:
#             if i != j:
#                 src, tar = 'data/' + domains[i], 'data/' + domains[j]
#                 src_domain, tar_domain = scipy.io.loadmat(src), scipy.io.loadmat(tar)
#                 Xs, Ys, Xt, Yt = src_domain['feas'], src_domain['label'], tar_domain['feas'], tar_domain['label']
#                 tca = TCA(kernel_type='linear', dim=30, lamb=1, gamma=1)
#                 acc, ypre = tca.fit_predict(Xs, Ys, Xt, Yt)
#                 print(acc)
#                 # It should print 0.910828025477707

In [3]:
# Compute MMD (maximum mean discrepancy) using numpy and scikit-learn.
from sklearn import metrics


def mmd_linear(X, Y):
    """MMD using linear kernel (i.e., k(x,y) = <x,y>)
    Note that this is not the original linear MMD, only the reformulated and faster version.
    The original version is:
        def mmd_linear(X, Y):
            XX = np.dot(X, X.T)
            YY = np.dot(Y, Y.T)
            XY = np.dot(X, Y.T)
            return XX.mean() + YY.mean() - 2 * XY.mean()
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Returns:
        [scalar] -- [MMD value]
    """
    delta = X.mean(0) - Y.mean(0)
    return delta.dot(delta.T)


def mmd_rbf(X, Y, gamma=1.0):
    """MMD using rbf (gaussian) kernel (i.e., k(x,y) = exp(-gamma * ||x-y||^2 / 2))
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Keyword Arguments:
        gamma {float} -- [kernel parameter] (default: {1.0})
    Returns:
        [scalar] -- [MMD value]
    """
    XX = metrics.pairwise.rbf_kernel(X, X, gamma)
    YY = metrics.pairwise.rbf_kernel(Y, Y, gamma)
    XY = metrics.pairwise.rbf_kernel(X, Y, gamma)
    return XX.mean() + YY.mean() - 2 * XY.mean()


def mmd_poly(X, Y, degree=2, gamma=1, coef0=0):
    """MMD using polynomial kernel (i.e., k(x,y) = (gamma <X, Y> + coef0)^degree)
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Keyword Arguments:
        degree {int} -- [degree] (default: {2})
        gamma {int} -- [gamma] (default: {1})
        coef0 {int} -- [constant item] (default: {0})
    Returns:
        [scalar] -- [MMD value]
    """
    XX = metrics.pairwise.polynomial_kernel(X, X, degree, gamma, coef0)
    YY = metrics.pairwise.polynomial_kernel(Y, Y, degree, gamma, coef0)
    XY = metrics.pairwise.polynomial_kernel(X, Y, degree, gamma, coef0)
    return XX.mean() + YY.mean() - 2 * XY.mean()

In [4]:
# 读取数据
def readimg(name):
    img = gdal.Open(os.path.join(filepath, name))
    img_b = img.GetRasterBand(1).ReadAsArray()
    img_g = img.GetRasterBand(2).ReadAsArray()
    img_r = img.GetRasterBand(3).ReadAsArray()
    img = np.dstack((img_b,img_g,img_r))
    print(img.shape)
    return img
# 数据分布可视化
def plotDist(data,name='',range=None):
    plt.hist(data[:,2], bins=256, facecolor='r', edgecolor='r', alpha=0.2)#bins=256
    plt.hist(data[:,1], bins=256, facecolor='g', edgecolor='g', alpha=0.2)
    plt.hist(data[:,0], bins=256, facecolor='b', edgecolor='b', alpha=0.2)
    plt.title(name+" distribution")
    plt.show()

In [5]:
def gendata(data,rate,num_0,num_1,num_2,num_3):
    X = data[:,:]
    label_0=np.zeros((num_0))
    label_1=np.ones((num_1))
    label_2=np.ones((num_2))+1
    label_3=np.ones((num_3))+2
    labels=np.concatenate((label_0,label_1,label_2,label_3),axis=0)
    labels=labels.reshape(labels.shape[0],1)
    train_dataset=np.concatenate((X,labels),axis=1)
    np.random.shuffle(train_dataset)
    train_dataset=train_dataset[:,:]
    #split train_data and test_data
    X=train_dataset[:,:3]
    labels=train_dataset[:,-1:]
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=rate)
    print("X_train shape:",X_train.shape)
    print("X_test shape:",X_test.shape)
    print("y_train shape:",y_train.shape)
    print("y_test shape:",y_test.shape)
    return X_train,X_test,y_train,y_test

In [6]:
# 获取标记数据的位置
def location(data):
    locat = np.zeros((data.shape[0],2),dtype=np.int)
    locat[:,0] = data[:,1]
    locat[:,1] = data[:,0]
    return locat

In [7]:
# 抽取逐年的数据
def sample(data,locat):
    sample = np.zeros((locat.shape[0],3),dtype=np.uint8)
    for i in range(locat.shape[0]):
        sample[i,:] = data[locat[i,0],locat[i,1],:]
    return sample

In [8]:
filepath = r"E:\project\images\researchImage\region"

In [9]:
data07norm=np.reshape(readimg("070727.tif"),(-1,3)).astype('float')/255
data10norm=np.reshape(readimg("100818.tif"),(-1,3)).astype('float')/255

(5415, 5451, 3)
(5415, 5451, 3)


In [16]:
labelData = np.loadtxt(open(r"E:\project\images\researchImage\region\label07.csv",'r'),delimiter=',',skiprows=21,dtype=np.int)

In [18]:
labelData[:5,:]

array([[     611,      525, 13223027,  3774935,       32,      118,
              39,       65,       80,      254],
       [     613,      526, 13223030,  3774934,       32,      118,
              37,       63,       78,      254],
       [     612,      526, 13223029,  3774934,       32,      118,
              39,       65,       80,      254],
       [     615,      527, 13223032,  3774933,       32,      118,
              42,       63,       80,      254],
       [     614,      527, 13223031,  3774933,       32,      118,
              36,       62,       77,      254]])

In [27]:
locat = location(labelData)

In [30]:
sample07 = sample(data07,locat)
sample10 = sample(data10,locat)
sample11 = sample(data11,locat)
sample13 = sample(data13,locat)

In [31]:
print("07 and 10 MMD:",mmd_linear(sample07,sample10))
print("07 and 11 MMD:",mmd_linear(sample07,sample11))
print("07 and 13 MMD:",mmd_linear(sample07,sample13))

07 and 10 MMD: 2348.8729608872936
07 and 11 MMD: 184.23993571847888
07 and 13 MMD: 794.7278196538184


In [33]:
X07_train,X07_test,y07_train,y07_test = gendata(sample07,0.2,231751,257769,251136,223094)

X_train shape: (771000, 3)
X_test shape: (192750, 3)
y_train shape: (771000, 1)
y_test shape: (192750, 1)


In [34]:
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X07_train,y07_train.ravel())
score = clf.score(X07_test,y07_test.ravel())
print('the score is :', score)

the score is : 0.897473411154345


In [4]:

# if __name__ == '__main__':
#     domains = ['caltech.mat', 'amazon.mat', 'webcam.mat', 'dslr.mat']
#     for i in [2]:
#         for j in [3]:
#             if i != j:
#                 src, tar = 'data/' + domains[i], 'data/' + domains[j]
#                 src_domain, tar_domain = scipy.io.loadmat(src), scipy.io.loadmat(tar)
#                 Xs, Ys, Xt, Yt = src_domain['feas'], src_domain['label'], tar_domain['feas'], tar_domain['label']
#                 tca = TCA(kernel_type='linear', dim=30, lamb=1, gamma=1)
#                 acc, ypre = tca.fit_predict(Xs, Ys, Xt, Yt)
#                 print(acc)
#                 # It should print 0.910828025477707

FileNotFoundError: [Errno 2] No such file or directory: 'data/webcam.mat'

In [7]:
label=cv2.imread(r"E:\project\images\researchImage\images\cut\label\17_12_11.tif",0)
data15=cv2.imread(r"E:\project\images\researchImage\images\cut\15\12_11.tif")
imgre=np.reshape(data15,(-1,3))



In [10]:
Ys=np.reshape(label,(65536))

In [9]:
data17=cv2.imread(r"E:\project\images\researchImage\images\cut\17\12_11.tif")
Xt=imgre
Xs=data17.reshape((-1,3))

In [11]:
clf=RandomForestClassifier(n_estimators=50)
clf.fit(Xs,Ys.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
Yt = clf.predict(Xt)


In [15]:
tca = TCA(kernel_type='linear', dim=10, lamb=1, gamma=1)
acc, ypre = tca.fit_predict(Xs, Ys, Xt, Yt)

TypeError: ufunc 'true_divide' output (typecode 'd') could not be coerced to provided output parameter (typecode 'B') according to the casting rule ''same_kind''